In [ ]:
!pip install langgraph langsmith

In [ ]:
! pip install langchain langchain_groq langchain_community

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
print(GOOGLE_API_KEY)
langsmith_api_key=userdata.get('langsmith_api_key')
print(langsmith_api_key)
groq_api_key=userdata.get('groq_api_key')
print(groq_api_key)

In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "CourseLanggraph"
os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm=ChatGroq( groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

## start building chatbot using langgraph


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages

In [ ]:
class State(TypedDict):
  # mesages have the type "list".the 'add_messages function
  # in the anntattion defines how this state key shoulf be updated
  # (in this case , it appends messages to the list , rather than overwriting them)
  messages:Annotated[list,add_messages]
graph_builder=StateGraph(State)

In [ ]:
graph_builder

In [ ]:
def chatbot(State:State):
  return {"messages":llm.invoke(State['messages'])}

In [ ]:
graph_builder.add_node("chatbot",chatbot)

In [ ]:
graph_builder

In [ ]:
graph_builder.add_edge(START,"chatbot")
graph_builder.add_edge("chatbot",END)

In [ ]:
graph=graph_builder.compile()

In [ ]:
from IPython.display import display,Image
try:
  display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
  pass


In [ ]:
while True:
  user_input=input("user.")
  if user_input.lower() in ["quit","q"]:
    print("GOOD BYE")
    break
  for event in graph.stream({'messages':("user",user_input)}):
    print(event.values())
    for value in event.values():
      print(value['messages'])
      print("Assistant:",value["messages"].content)